<a href="https://colab.research.google.com/github/arsendoinychko/Cp-decomposition/blob/main/Untggg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time

batch_size = 64
learning_rate = 0.001
num_epochs = 2

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    start_time = time.time()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0
    epoch_duration = time.time() - start_time
    print(f'Epoch [{epoch + 1}/{num_epochs}] completed in {epoch_duration:.2f} seconds')

print('Finished Training')


correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

torch.save(model.state_dict(), 'cnn_mnist_model.pth')

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:01<00:00, 6240027.38it/s] 


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1163915.73it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 8523962.49it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 851420.28it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Epoch [1/2], Step [100/938], Loss: 0.4910
Epoch [1/2], Step [200/938], Loss: 0.1123
Epoch [1/2], Step [300/938], Loss: 0.0954
Epoch [1/2], Step [400/938], Loss: 0.0815
Epoch [1/2], Step [500/938], Loss: 0.0788
Epoch [1/2], Step [600/938], Loss: 0.0585
Epoch [1/2], Step [700/938], Loss: 0.0563
Epoch [1/2], Step [800/938], Loss: 0.0641
Epoch [1/2], Step [900/938], Loss: 0.0555
Epoch [1/2] completed in 140.60 seconds
Epoch [2/2], Step [100/938], Loss: 0.0371
Epoch [2/2], Step [200/938], Loss: 0.0376
Epoch [2/2], Step [300/938], Loss: 0.0353
Epoch [2/2], Step [400/938], Loss: 0.0443
Epoch [2/2], Step [500/938], Loss: 0.0326
Epoch [2/2], Step [600/938], Loss: 0.0445
Epoch [2/2], Step [700/938], Loss: 0.0363
Epoch [2/2], Step [800/938], Loss: 0.0393
Epoch [2/2], Step [900/938], Loss: 0.0417
Epoch [2/2] completed in 144.55 seconds
Finished Training
Accuracy of the network on the 10000 test images: 98.72%


In [ ]:
pip install tensorly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.7/229.7 kB 1.7 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tensorly.decomposition import parafac
import tensorly as tl

def cp_decomposition_conv_layer(layer, rank):

    S = layer.in_channels
    T = layer.out_channels
    rank = S*T/(S+T)
    rank = round(rank)
    print(rank)

    weights,(last, first, vertical, horizontal) = parafac(layer.weight.data, rank=rank, init='svd')

    pointwise_s_to_r_layer = torch.nn.Conv2d(in_channels=first.shape[0], \
            out_channels=first.shape[1], kernel_size=1, stride=1, padding=0,
            dilation=layer.dilation, bias=False)

    depthwise_vertical_layer = torch.nn.Conv2d(in_channels=vertical.shape[1],
            out_channels=vertical.shape[1], kernel_size=(vertical.shape[0], 1),
            stride=1, padding=(layer.padding[0], 0), dilation=layer.dilation,
            groups=vertical.shape[1], bias=False)

    depthwise_horizontal_layer = \
        torch.nn.Conv2d(in_channels=horizontal.shape[1], \
            out_channels=horizontal.shape[1],
            kernel_size=(1, horizontal.shape[0]), stride=layer.stride,
            padding=(0, layer.padding[0]),
            dilation=layer.dilation, groups=horizontal.shape[1], bias=False)

    pointwise_r_to_t_layer = torch.nn.Conv2d(in_channels=last.shape[1], \
            out_channels=last.shape[0], kernel_size=1, stride=1,
            padding=0, dilation=layer.dilation, bias=True)

    pointwise_r_to_t_layer.bias.data = layer.bias.data

    depthwise_horizontal_layer.weight.data = \
        torch.transpose(horizontal, 1, 0).unsqueeze(1).unsqueeze(1)
    depthwise_vertical_layer.weight.data = \
        torch.transpose(vertical, 1, 0).unsqueeze(1).unsqueeze(-1)
    pointwise_s_to_r_layer.weight.data = \
        torch.transpose(first, 1, 0).unsqueeze(-1).unsqueeze(-1)
    pointwise_r_to_t_layer.weight.data = last.unsqueeze(-1).unsqueeze(-1)

    new_layers = [pointwise_s_to_r_layer, depthwise_vertical_layer, \
                    depthwise_horizontal_layer, pointwise_r_to_t_layer]

    return nn.Sequential(*new_layers)


In [ ]:
layer = dict(model.named_children())['conv2']

In [ ]:
print(model)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=3136, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=10, bias=True)
)


In [ ]:
tl.set_backend('pytorch')

In [ ]:
new_layers = cp_decomposition_conv_layer(layer, 10)

21


In [ ]:
new_layers

Sequential(
  (0): Conv2d(32, 21, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): Conv2d(21, 21, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), groups=21, bias=False)
  (2): Conv2d(21, 21, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), groups=21, bias=False)
  (3): Conv2d(21, 64, kernel_size=(1, 1), stride=(1, 1))
)

In [ ]:
setattr(model, 'conv2', nn.Sequential(*new_layers))

In [ ]:
print(model)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Sequential(
    (0): Conv2d(32, 21, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): Conv2d(21, 21, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), groups=21, bias=False)
    (2): Conv2d(21, 21, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), groups=21, bias=False)
    (3): Conv2d(21, 64, kernel_size=(1, 1), stride=(1, 1))
  )
  (fc1): Linear(in_features=3136, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=10, bias=True)
)


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the network on the 10000 test images: 98.42%


In [ ]:
for epoch in range(num_epochs):
    start_time = time.time()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0
    epoch_duration = time.time() - start_time
    print(f'Epoch [{epoch + 1}/{num_epochs}] completed in {epoch_duration:.2f} seconds')

correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

Epoch [1/2], Step [100/938], Loss: 0.0284
Epoch [1/2], Step [200/938], Loss: 0.0250
Epoch [1/2], Step [300/938], Loss: 0.0307
Epoch [1/2], Step [400/938], Loss: 0.0274
Epoch [1/2], Step [500/938], Loss: 0.0284
Epoch [1/2], Step [600/938], Loss: 0.0248
Epoch [1/2], Step [700/938], Loss: 0.0188
Epoch [1/2], Step [800/938], Loss: 0.0354
Epoch [1/2], Step [900/938], Loss: 0.0231
Epoch [1/2] completed in 154.62 seconds
Epoch [2/2], Step [100/938], Loss: 0.0138
Epoch [2/2], Step [200/938], Loss: 0.0172
Epoch [2/2], Step [300/938], Loss: 0.0174
Epoch [2/2], Step [400/938], Loss: 0.0153
Epoch [2/2], Step [500/938], Loss: 0.0174
Epoch [2/2], Step [600/938], Loss: 0.0166
Epoch [2/2], Step [700/938], Loss: 0.0185
Epoch [2/2], Step [800/938], Loss: 0.0206
Epoch [2/2], Step [900/938], Loss: 0.0240
Epoch [2/2] completed in 157.64 seconds
Accuracy of the network on the 10000 test images: 98.97%


In [ ]:
pip install torchprofile

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchprofile

dummy_input = torch.randn(1, 1, 28, 28)

# Обчислення FLOPs
flops = torchprofile.profile_macs(model, dummy_input)

# Перетворення в GFLOPs
gflops = flops / 1e9
gflops =gflops*1000

print(f'FLOPs in convolutions (Giga): {gflops:.2f}')

FLOPs in convolutions (Giga): 3.57


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time

# Гіперпараметри
batch_size = 64
learning_rate = 0.001
num_epochs = 2

# Вибір пристрою (GPU, якщо доступний, або CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Завантаження та нормалізація набору даних MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Визначення конволюційної нейронної мережі
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)  # Розгортаємо тензор для повнозв'язного шару
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Ініціалізація моделі, функції втрат та оптимізатора
model = CNN().to(device)  # Перенесення моделі на GPU
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Навчання моделі
for epoch in range(num_epochs):
    start_time = time.time()  # Початок відліку часу
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)  # Перенесення даних на GPU

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # Кожні 100 міні-батчів
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0
    epoch_duration = time.time() - start_time  # Кінець відліку часу
    print(f'Epoch [{epoch + 1}/{num_epochs}] completed in {epoch_duration:.2f} seconds')

print('Finished Training')

# Тестування моделі
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Перенесення даних на GPU
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

# Збереження моделі
torch.save(model.state_dict(), 'cnn_mnist_model.pth')


Using device: cpu
Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 43493286.76it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1688137.66it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 13631078.79it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 8604574.87it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/2], Step [100/938], Loss: 0.6337
Epoch [1/2], Step [200/938], Loss: 0.1476
Epoch [1/2], Step [300/938], Loss: 0.0933
Epoch [1/2], Step [400/938], Loss: 0.0831
Epoch [1/2], Step [500/938], Loss: 0.0724
Epoch [1/2], Step [600/938], Loss: 0.0703
Epoch [1/2], Step [700/938], Loss: 0.0688
Epoch [1/2], Step [800/938], Loss: 0.0653
Epoch [1/2], Step [900/938], Loss: 0.0573
Epoch [1/2] completed in 144.91 seconds
Epoch [2/2], Step [100/938], Loss: 0.0341
Epoch [2/2], Step [200/938], Loss: 0.0392
Epoch [2/2], Step [300/938], Loss: 0.0413
Epoch [2/2], Step [400/938], Loss: 0.0421
Epoch [2/2], Step [500/938], Loss: 0.0420
Epoch [2/2], Step [600/938], Loss: 0.0365
Epoch [2/2], Step [700/938], Loss: 0.0404
Epoch [2/2], Step [800/938], Loss: 0.0465
Epoch [2/2], Step [900/938], Loss: 0.0405
Epoch [2/2] completed in 157.55 seconds
Finished Training
Accuracy of the network on the 10000 test images: 98.98%


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchprofile

dummy_input = torch.randn(1, 1, 28, 28)

# Обчислення FLOPs
flops = torchprofile.profile_macs(model, dummy_input)

# Перетворення в GFLOPs
gflops = flops / 1e9
gflops =gflops*1000

print(f'FLOPs in convolutions (Giga): {gflops:.2f}')

FLOPs in convolutions (Giga): 3.57


In [ ]:
import torch
# Перевірка доступності GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cpu


In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import tensorly as tl
from tensorly.decomposition import tucker, partial_tucker

In [ ]:
pip install tensorly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.7/229.7 kB 4.2 MB/s eta 0:00:00


In [ ]:
core, [last, first] = partial_tucker(tensor, modes=[0,1], ranks=5, init='svd')

TypeError: partial_tucker() got an unexpected keyword argument 'ranks'

In [ ]:
tensor = torch.randn(2, 3, 4, 5)

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import tensorly as tl
from tensorly.decomposition import tucker, partial_tucker

def tucker_decomp(layer, rank):
    W = layer.weight.data

    core, factors = partial_tucker(W, modes=[0,1], rank=rank, init='svd')

    last, first, _ = factors

    first_layer = nn.Conv2d(in_channels=first.shape[0],
                                       out_channels=first.shape[1],
                                       kernel_size=1,
                                       padding=0,
                                       bias=False)

    core_layer = nn.Conv2d(in_channels=core.shape[1],
                                       out_channels=core.shape[0],
                                       kernel_size=layer.kernel_size,
                                       stride=layer.stride,
                                       padding=layer.padding,
                                       dilation=layer.dilation,
                                       bias=False)

    last_layer = nn.Conv2d(in_channels=last.shape[1],
                                       out_channels=last.shape[0],
                                       kernel_size=1,
                                       padding=0,
                                       bias=True)

    if layer.bias is not None:
        last_layer.bias.data = layer.bias.data

    fk = first.t_().unsqueeze_(-1).unsqueeze_(-1)
    lk = last.unsqueeze_(-1).unsqueeze_(-1)

    first_layer.weight.data = fk
    last_layer.weight.data = lk
    core_layer.weight.data = core

    new_layers = [first_layer, core_layer, last_layer]
    return new_layers



In [ ]:
new_layers = tucker_decomp(layer, 10)

IndexError: tuple index out of range

In [ ]:
W = layer.weight.data

In [ ]:
W.shape

torch.Size([64, 32, 3, 3])

In [ ]:
core, factors = partial_tucker(W, modes=[0,1], rank=30, init='svd')

In [ ]:
factors

[tensor(0.4314), tensor(0.4311), tensor(0.4311)]

In [ ]:
last, first, _ = factors

In [ ]:
first.shape

torch.Size([])

In [ ]:
import tensorly as tl
from tensorly.decomposition import partial_tucker
import torch

# Створимо приклад 4D тензора
shape = (10, 8, 5, 5)  # Приклад розмірів (out_channels, in_channels, height, width)
tensor_4d = torch.randn(*shape)  # Перетворення на тензор PyTorch

# Виконати частковий Tucker-розклад
core, factors = partial_tucker(tensor_4d, modes=[0, 1], rank=[3, 4], init='svd')

# Вивести розміри отриманих результатів
print(f"Core shape: {core.shape if isinstance(core, torch.Tensor) else 'Not a tensor'}")
print(f"Factor shapes: {[factor.shape if isinstance(factor, torch.Tensor) else 'Not a tensor' for factor in factors]}")



Core shape: Not a tensor
Factor shapes: [torch.Size([]), torch.Size([]), torch.Size([]), torch.Size([]), torch.Size([]), torch.Size([])]


In [ ]:
W = layer.weight.data.numpy()  # Перетворення на numpy для використання в TensorLy

    # Виконання часткового Tucker-розкладу з двома модами
core, factors = partial_tucker(W, modes=[0, 1], rank=10, init='svd')

    # Отримання двох факторів
last, first = factors  # Порядок залежить від partial_tucker

    # Перевірка розмірів факторів
print(f"first shape: {first.shape}")
print(f"last shape: {last.shape}")
print(f"core shape: {core.shape}")

TypeError: moveaxis() received an invalid combination of arguments - got (numpy.ndarray, int, int), but expected one of:
 * (Tensor input, int source, int destination)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, int, int)
 * (Tensor input, tuple of ints source, tuple of ints destination)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !int!, !int!)


In [ ]:
print(f"Factors: {factors}")

# Перевірка форми кожного фактора
for i, factor in enumerate(factors):
    print(f"Factor {i} shape: {factor.shape}")

Factors: [tensor(0.7286), tensor(0.7283), tensor(0.7283)]
Factor 0 shape: torch.Size([])
Factor 1 shape: torch.Size([])
Factor 2 shape: torch.Size([])


In [ ]:
print(f"W shape: {W.shape}")

W shape: torch.Size([64, 32, 3, 3])
